<a href="https://colab.research.google.com/github/rahulsm27/Colab_practicse/blob/main/DQN%2BPER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium
!pip install -q swig
!pip install -q gymnasium[box2d]


In [18]:
!pip install sumtree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.9 MB/s eta 0:00:00


In [22]:
from sumtree import SumTree


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
import random
from collections import namedtuple, deque
import numpy as np
from numpy.random import choice
import torch
import operator
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, experiences_per_sampling, seed, compute_weights):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            experiences_per_sampling (int): number of experiences to sample during a sampling iteration
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.experiences_per_sampling = experiences_per_sampling

        self.alpha = 0.5
        self.alpha_decay_rate = 0.99
        self.beta = 0.5
        self.beta_growth_rate = 1.001
        self.seed = random.seed(seed)
        self.compute_weights = compute_weights
        self.experience_count = 0

        self.experience = namedtuple("Experience",
            field_names=["state", "action", "reward", "next_state", "done"])
        self.data = namedtuple("Data",
            field_names=["priority", "probability", "weight","index"])

        indexes = []
        datas = []
        for i in range(buffer_size):
            indexes.append(i)
            d = self.data(0,0,0,i)
            datas.append(d)

        self.memory = {key: self.experience for key in indexes}
        self.memory_data = {key: data for key,data in zip(indexes, datas)}
        self.sampled_batches = []
        self.current_batch = 0
        self.priorities_sum_alpha = 0
        self.priorities_max = 1
        self.weights_max = 1



    def update_priorities(self, tds, indices):
        for td, index in zip(tds, indices):
            N = min(self.experience_count, self.buffer_size)

            updated_priority = td[0]
            if updated_priority > self.priorities_max:
                self.priorities_max = updated_priority

            if self.compute_weights:
                updated_weight = ((N * updated_priority)**(-self.beta))/self.weights_max
                if updated_weight > self.weights_max:
                    self.weights_max = updated_weight
            else:
                updated_weight = 1

            old_priority = self.memory_data[index].priority
            self.priorities_sum_alpha += updated_priority**self.alpha - old_priority**self.alpha
            updated_probability = td[0]**self.alpha / self.priorities_sum_alpha
            data = self.data(updated_priority, updated_probability, updated_weight, index)
            self.memory_data[index] = data

    def update_memory_sampling(self):
        """Randomly sample X batches of experiences from memory."""
        # X is the number of steps before updating memory
        self.current_batch = 0
        values = list(self.memory_data.values())
        random_values = random.choices(self.memory_data,
                                       [data.probability for data in values],
                                       k=self.experiences_per_sampling)
        self.sampled_batches = [random_values[i:i + self.batch_size]
                                    for i in range(0, len(random_values), self.batch_size)]

    def update_parameters(self):
        self.alpha *= self.alpha_decay_rate
        self.beta *= self.beta_growth_rate
        if self.beta > 1:
            self.beta = 1
        N = min(self.experience_count, self.buffer_size)
        self.priorities_sum_alpha = 0
        sum_prob_before = 0
        for element in self.memory_data.values():
            sum_prob_before += element.probability
            self.priorities_sum_alpha += element.priority**self.alpha
        sum_prob_after = 0
        for element in self.memory_data.values():
            probability = element.priority**self.alpha / self.priorities_sum_alpha
            sum_prob_after += probability
            weight = 1
            if self.compute_weights:
                weight = ((N *  element.probability)**(-self.beta))/self.weights_max
            d = self.data(element.priority, probability, weight, element.index)
            self.memory_data[element.index] = d
        print("sum_prob before", sum_prob_before)
        print("sum_prob after : ", sum_prob_after)

    def push(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        self.experience_count += 1
        index = self.experience_count % self.buffer_size

        if self.experience_count > self.buffer_size:
            temp = self.memory_data[index]
            self.priorities_sum_alpha -= temp.priority**self.alpha
            if temp.priority == self.priorities_max:
                self.memory_data[index]._replace(priority = 0)
                self.priorities_max = max(self.memory_data.items(), key=operator.itemgetter(1))[1][0]
            if self.compute_weights:
                if temp.weight == self.weights_max:
                    self.memory_data[index]._replace(weight = 0)
                    self.weights_max = max(self.memory_data.items(), key=operator.itemgetter(2))[1][2]

        priority = self.priorities_max
        weight = self.weights_max
        self.priorities_sum_alpha += priority ** self.alpha
        probability = priority ** self.alpha / self.priorities_sum_alpha
        e = self.experience(state, action, reward, next_state, done)
        self.memory[index] = e
        d = self.data(priority, probability, weight, index)
        self.memory_data[index] = d

    def sample(self):
        sampled_batch = self.sampled_batches[self.current_batch]
        self.current_batch += 1
        experiences = []
        weights = []
        indices = []

        for data in sampled_batch:
            experiences.append(self.memory.get(data.index))
            weights.append(data.weight)
            indices.append(data.index)

        states = torch.from_numpy(
            np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(
            np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(
            np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(
            np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(
            np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones, weights, indices)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import gymnasium as gym
import math
import random
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import imageio
from gym.envs import box2d

from IPython.display import HTML, display
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

import os
#### If you encounter error related to "libiomp5md.dll", you can try the code in the next line.
#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:  ", device)
# ==================================================================
#  Define Basic Classes
#  - Finish get_state_values() function in class DQN
#  - Finish forward() function and get_state_values() function in DuelingDQN class for bonus
# ==================================================================

#### Definition of Standard Experience Replay Buffer
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'terminated'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

#### Definition of Standard DQN network
class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.L1 = nn.Linear(n_observations, 64)
        self.L2 = nn.Linear(64, 64)
        self.L3 = nn.Linear(64, n_actions)

    def forward(self, x):
        x = F.relu(self.L1(x))
        x = F.relu(self.L2(x))
        return self.L3(x)

    def get_state_action_values(self, state_batch, action_batch):
        q_values = self(state_batch)
        row_index = torch.arange(0, state_batch.shape[0])
        selected_actions_q_values = q_values[row_index, action_batch]
        return selected_actions_q_values

    def get_state_values(self, state_batch):
        q_values = self(state_batch)
        # Get the maximum Q value for each state
        max_q_values, _ = q_values.max(dim=1)
        return max_q_values

#### Definition of Dueling Networkss
class DuelingDQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DuelingDQN, self).__init__()
        # self.feature_layer = nn.Linear(n_observations, 64)

        # Value stream
        self.value_stream = nn.Sequential(
            nn.Linear(n_observations, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        # Advantage stream
        self.advantage_stream = nn.Sequential(
            nn.Linear(n_observations, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, n_actions)
        )

    def forward(self, x):
        value = self.value_stream(x)
        advantages = self.advantage_stream(x)

        # Subtracting the mean of the advantages to make them have zero mean
        # This ensures the Q values are properly centered around the value function
        Q_values = value + (advantages - advantages.mean(dim=1, keepdim=True))

        return Q_values

    # Get Q(s,a)
    def get_state_action_values(self, state_batch, action_batch):
        q_values = self(state_batch)
        row_index = torch.arange(0, state_batch.shape[0])
        selected_actions_q_values = q_values[row_index, action_batch]
        return selected_actions_q_values

    def get_state_values(self, state_batch):
        # Forward pass through the network to get the Q values for each action
        Q_values = self.forward(state_batch)

        # Find the maximum Q value among all actions for each state
        max_state_values, _ = Q_values.max(dim=1)

        return max_state_values




#=========================================================================



# ===================================
#  Hyperparameters
# ===================================
BATCH_SIZE = 64
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.003
EPS_DECAY = 1000
TAU = 0.002
LR = 1e-4
# ==================================


# =====================================================================
#  Initialize Environment and Networks
# =====================================================================
env = gym.make('LunarLander-v2')
#### Get the dimension of action and state
n_actions = env.action_space.n
n_observations = env.observation_space.shape[0]


#### Initilize DQN/DDQN Networks and optimizer
policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)


#### Initilize Dueling Networks and optimizer
policy_net_duel = DuelingDQN(n_observations, n_actions).to(device)
target_net_duel = DuelingDQN(n_observations, n_actions).to(device)
target_net_duel.load_state_dict(policy_net_duel.state_dict())
## Only update the parameter for the policy network
optimizer_duel = optim.AdamW(policy_net_duel.parameters(), lr=LR, amsgrad=True)

#### Initizalize Experience Replay Buffer
memory = ReplayMemory(10000)

#### Initizalize Priority Experience Replay Buffer
        # Replay memory

# prioritized experience replay
UPDATE_NN_EVERY = 1
UPDATE_MEM_EVERY = 20          # how often to update the priorities
UPDATE_MEM_PAR_EVERY = 3000     # how often to update the hyperparameters
EXPERIENCES_PER_SAMPLING = math.ceil(BATCH_SIZE * UPDATE_MEM_EVERY / UPDATE_NN_EVERY)


pmemory = ReplayBuffer(action_size=4, buffer_size=int(1e5), batch_size=64, experiences_per_sampling=EXPERIENCES_PER_SAMPLING, seed=0, compute_weights =False)

# ======================================================================



# ===============================================================================================================
#  Define Main Algorithms
#  - Finish the update of optimize_model_DQN() amd optimize_model_DDQN()
#  - Finish the update of optimize_model_DN() for bonus
# ===============================================================================================================

#### Implementation of vanilla DQN
def optimize_model_DQN():
    if len(memory) < BATCH_SIZE:
        return

    states, actions, rewards, next_states, terminateds = zip(*memory.sample(BATCH_SIZE))

    # Convert to tensors
    state_batch = torch.tensor(np.array(states), device=device, dtype=torch.float)
    action_batch = torch.tensor(actions, device=device).unsqueeze(-1) # Actions are used as indices
    reward_batch = torch.tensor(rewards, device=device, dtype=torch.float)
    next_state_batch = torch.tensor(np.array(next_states), device=device, dtype=torch.float)
    terminated_batch = torch.tensor(terminateds, device=device, dtype=torch.float)

    # Compute Q(s, a)
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute max_a Q(s', a) for all next states.
    next_state_values = target_net(next_state_batch).max(1)[0].detach()

    # If next state is terminal, then there is no next Q value
    next_state_values = next_state_values * (1 - terminated_batch)

    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute loss
    loss = F.mse_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)  # Clipping gradients to avoid explosion
    optimizer.step()


#### Implementation of Double DQN
def optimize_model_DDQN():
    if len(memory) < BATCH_SIZE:
        return

    states, actions, rewards, next_states, terminateds = zip(*memory.sample(BATCH_SIZE))

    # Convert to tensors
    state_batch = torch.tensor(np.array(states), device=device, dtype=torch.float)
    action_batch = torch.tensor(actions, device=device).unsqueeze(-1)  # Actions are used as indices
    reward_batch = torch.tensor(rewards, device=device, dtype=torch.float)
    next_state_batch = torch.tensor(np.array(next_states), device=device, dtype=torch.float)
    terminated_batch = torch.tensor(terminateds, device=device, dtype=torch.float)

    # Compute Q(s, a) using the policy network
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Select the best action in the next state using the policy network (action selection)
    next_state_actions = policy_net(next_state_batch).max(1)[1].unsqueeze(-1)

    # Evaluate the selected action's Q value using the target network (action evaluation)
    next_state_values = target_net(next_state_batch).gather(1, next_state_actions).squeeze(-1)

    # If next state is terminal, then there is no next Q value
    next_state_values = next_state_values * (1 - terminated_batch)

    # Compute the expected Q values
    expected_state_action_values = reward_batch + (GAMMA * next_state_values)

    # Compute loss
    loss = F.mse_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


#### Implementation of Double DQN + Dueling Network
def optimize_model_DN():
    if len(memory) < BATCH_SIZE:
        return

    # Extract transitions from the replay memory
    states, actions, rewards, next_states, terminateds = zip(*memory.sample(BATCH_SIZE))

    # Convert arrays into tensors
    state_batch = torch.tensor(np.array(states), device=device, dtype=torch.float)
    action_batch = torch.tensor(actions, device=device).unsqueeze(-1)
    reward_batch = torch.tensor(rewards, device=device, dtype=torch.float)
    next_state_batch = torch.tensor(np.array(next_states), device=device, dtype=torch.float)
    terminated_batch = torch.tensor(terminateds, device=device, dtype=torch.float)

    # Compute Q(s, a) using the policy network
    state_action_values = policy_net_duel(state_batch).gather(1, action_batch)

    # Determine the best action for each next state using the policy network
    next_state_actions = policy_net_duel(next_state_batch).max(1)[1].unsqueeze(-1)

    # Evaluate the selected actions' Q values using the target network
    next_state_values = target_net_duel(next_state_batch).gather(1, next_state_actions).squeeze(-1)

    # Apply terminal mask to ensure no next state value is considered for terminal states
    next_state_values = next_state_values * (1 - terminated_batch)

    # Compute the expected Q values
    expected_state_action_values = reward_batch + (GAMMA * next_state_values)

    # Compute loss
    loss = F.mse_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer_duel.zero_grad()
    loss.backward()
    optimizer_duel.step()
# ==============================================================================================================================

#### Implementation of Double DQN + Dueling Network + PriorityRelay
def optimize_model_DNP():
    if len(pmemory) < BATCH_SIZE:
        return


    # Extract transitions into separate arrays
    states, actions, rewards, next_states, terminateds, weights, indices = pmemory.sample()

    # Convert arrays into tensors
    state_batch = torch.tensor(np.array(states), device=device, dtype=torch.float)
    action_batch = torch.tensor(actions, device=device).unsqueeze(-1)
    reward_batch = torch.tensor(rewards, device=device, dtype=torch.float)
    next_state_batch = torch.tensor(np.array(next_states), device=device, dtype=torch.float)
    terminated_batch = torch.tensor(terminateds, device=device, dtype=torch.float)

    # Compute Q(s, a) using the policy network
    state_action_values = policy_net_duel(state_batch).gather(1, action_batch)

    # Determine the best action for each next state using the policy network
    next_state_actions = policy_net_duel(next_state_batch).max(1)[1].unsqueeze(-1)

    # Evaluate the selected actions' Q values using the target network
    next_state_values = target_net_duel(next_state_batch).gather(1, next_state_actions).squeeze(-1)

    # Apply terminal mask to ensure no next state value is considered for terminal states
    next_state_values = next_state_values * (1 - terminated_batch)

    # Compute the expected Q values
    expected_state_action_values = reward_batch + (GAMMA * next_state_values)

    # Compute TD errors for prioritized experience replay
    td_errors = (state_action_values - expected_state_action_values.unsqueeze(1)).abs().squeeze(-1)

    # Update priorities in the prioritized experience replay memory
    pmemory.update_priorities( td_errors.detach().cpu().numpy(),indices)

    # Compute loss weighted by importance sampling weights
    weighted_loss = (weights * F.mse_loss(state_action_values, expected_state_action_values.unsqueeze(1), reduction='none')).mean()

    # Optimize the model
    optimizer_duel.zero_grad()
    weighted_loss.backward()
    optimizer_duel.step()
# ==============================================================================================================================


# ===============================================================================================================
#  Main Train Loop
#  - Finish the epsilon greedy exploration
# ===============================================================================================================

#### Training Episodes
NUM_EPISODES = 2000

#### Training Loop. If the input algorithm == "DQN", it will utilize DQN to train.
#### Similarly, if the input algorithm == "DDQN", it will utilize DDQN to train. If the input algorithm == "DN", it will utilize Dueling Networks to train
def train_models(algorithm):
    episode_returns = []  # To store returns from each episode

    for iteration in range(NUM_EPISODES):
        current_episode_return = 0  # Reset the return for the current episode
        state, _ = env.reset()  # Reset the environment
        terminated = False
        truncated = False

        # Initialize time step (for updating every UPDATE_NN_EVERY steps)
        t_step_nn = 0
        # Initialize time step (for updating every UPDATE_MEM_PAR_EVERY steps)
        t_step_mem_par = 0
        # Initialize time step (for updating every UPDATE_MEM_EVERY steps)
        t_step_mem = 0



        while not (terminated or truncated):
            # Epsilon-Greedy Exploration
            eps = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * iteration / EPS_DECAY)
            if random.random() > eps:
                # Convert state to tensor
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                with torch.no_grad():
                    if algorithm in ["DQN", "DDQN"]:
                        action_values = policy_net(state_tensor)
                    elif algorithm == "DN" or algorithm == "DNP":
                        action_values = policy_net_duel(state_tensor)
                    action = action_values.max(1)[1].item()  # Choose the action with the highest Q-value
            else:
                action = random.randrange(n_actions)  # Choose a random action

            # Take action in the environment
            next_state, reward, terminated, truncated, _ = env.step(action)
            if not algorithm == 'DNP':
                memory.push(state, action, reward, next_state, terminated)  # Save the transition in memory
            else:
                pmemory.push(state, action, reward, next_state, terminated)
                        # Learn every UPDATE_NN_EVERY time steps.
                t_step_nn = (t_step_nn + 1) # UPDATE_NN_EVERY
                t_step_mem = (t_step_mem + 1) # UPDATE_MEM_EVERY
                t_step_mem_par = (t_step_mem_par + 1) # UPDATE_MEM_PAR_EVERY
                if t_step_mem_par == 0:
                    memory.update_parameters()
                if t_step_nn == 0:
                    # If enough samples are available in memory, get random subset and learn
                    if memory.experience_count > EXPERIENCES_PER_SAMPLING:
                      optimize_model_DN()

                if t_step_mem == 0:
                    memory.update_memory_sampling()



            current_episode_return += reward

            # Update the target network parameters
            if algorithm in ["DQN", "DDQN"]:
                for target_par0am, policy_param in zip(target_net.parameters(), policy_net.parameters()):
                    target_param.data.copy_(TAU * policy_param.data + (1.0 - TAU) * target_param.data)
            elif algorithm == "DN" or algorithm == "DNP":
                for target_param, policy_param in zip(target_net_duel.parameters(), policy_net_duel.parameters()):
                    target_param.data.copy_(TAU * policy_param.data + (1.0 - TAU) * target_param.data)

            if terminated or truncated:
                if iteration % 20 == 0:
                    print('Episode {},  score: {}'.format(iteration, current_episode_return))
                episode_returns.append(current_episode_return)
            else:
                state = next_state  # Update the state

            # Optimize the model according to the selected algorithm
            if algorithm == "DQN":
                optimize_model_DQN()
            elif algorithm == "DDQN":
                optimize_model_DDQN()
            elif algorithm == "DN":
                optimize_model_DN()



    plt.figure(figsize=(15, 7))
    plt.title('Training with ' + algorithm)
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    # plt.plot(episode_returns)

    # Plot the full episode returns without markers
    plt.plot(episode_returns, color='blue', alpha=0.5)

    # Create a subset of episode_returns with only the points after every 20 episodes
    marker_indices = [i for i in range(len(episode_returns)) if i % 20 == 0]
    marker_returns = [episode_returns[i] for i in marker_indices]

    # Plot the subset of episode returns with circular markers
    plt.plot(marker_indices, marker_returns, 'o', markersize=5, color='orange')

    plt.savefig("Training with " + algorithm)
    plt.show()

# ===============================================================================================================
#  Functions to generate videos for visulization
#  (This part is only for playing. You do not need to upload any video for homework)
#  To play with your own policy, you can save your policy network and put it into generate_videos(policy) as policy
#  You also need to complete a line in generate_videos(policy) to play.
#  This line is the same as the line you finishes in train_models(algorithm), choosing the best action based on your model.
# ===============================================================================================================
def generate_videos(policy):
    env = gym.make('LunarLander-v2', render_mode = 'rgb_array')
    n_actions = env.action_space.n
    n_observations = env.observation_space.shape[0]

    frames = []

    state, _ = env.reset()
    img = env.render()
    frames.append(img)
    terminated = False
    truncated = False
    while not (terminated or truncated):
        if policy == "random":
            action = np.random.randint(0, n_actions)
        else:
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            with torch.no_grad():
                # Example for using the DuelingDQN policy network
                action_values = policy_net_duel(state_tensor)
            action = action_values.max(1)[1].item()
        next_state, _, terminated, truncated, _ = env.step(action)
        img = env.render()
        frames.append(img)
        if terminated or truncated:
            break
        state = next_state
    imageio.mimsave(f'lunar_lander_{policy}.mp4', frames, fps=60)
# ===============================================================================================================




train_models("DNP")
generate_videos("DNP")


device:   cpu
Episode 0,  score: -99.7577656729626
Episode 20,  score: -547.468916823508
Episode 40,  score: -363.40281129923136
Episode 60,  score: -289.21218442716065
Episode 80,  score: -220.46230282724798
Episode 100,  score: -301.8867098237258
Episode 120,  score: -218.67333824275102
Episode 140,  score: -279.0619520598478
Episode 160,  score: -155.3835023164041
Episode 180,  score: -308.4364099998279
Episode 200,  score: -135.22359641821373
Episode 220,  score: -118.46021934294282
Episode 240,  score: -183.35165102848805
Episode 260,  score: -172.11376025899474
Episode 280,  score: -11.067817332777636
Episode 300,  score: -329.0281792306222
Episode 320,  score: -145.43008843203415
Episode 340,  score: -166.2317382615377
Episode 360,  score: -211.6041753045741
Episode 380,  score: -211.77670018416973
Episode 400,  score: -450.1961170697697
Episode 420,  score: -302.4083964982169
Episode 440,  score: -61.19628386183707
Episode 460,  score: -405.85159858770885
Episode 480,  score: -